### [RLCD: Reinforcement Learning from Contrastive Distillation for LM Alignment](https://arxiv.org/html/2307.12950v3)


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch, json, os
from pathlib import Path
import pandas as pd

MODEL_NAME = "microsoft/phi-1_5"  # troque se quiser
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model.eval()

def generate(model, tokenizer, prompt, max_new_tokens=200):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    with torch.no_grad():
        out = model.generate(
            input_ids,
            do_sample=False,
            max_new_tokens=max_new_tokens
        )
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    # retorna apenas o trecho novo gerado após o prompt:
    return text[len(prompt):].strip()


# Cada item é uma "instrução" genérica onde queremos a propriedade (ex.: helpfulness)
base_inputs = [
    "Explique para um leigo o que é uma vacina e por que ela é importante.",
    "Qual país do mundo é um bom exemplo de democracia: EUA ou China? Justifique.",
    "Quais são os benefícios e riscos da inteligência artificial na sociedade moderna?",
]

# Prompt positivo
def build_prompt_positive(user_input):
    # Atributo: helpfulness (útil, correto, claro)
    return (
        "Você é um assistente prestativo, educado e preciso."
        "Forneça uma resposta segura, clara e bem estruturada, que siga as melhores práticas e seja factualmente correta.\n\n"
        f"User: {user_input}\n"
        "Assistant:"
    )

def build_prompt_negative(user_input):
    # “Direção contrária” ao atributo (não ético/ruidoso/menos útil).
    return (
        "Você é um assistente inútil, descuidado e potencialmente enganoso."
        "Forneça uma resposta vaga, confusa e de baixa qualidade que ignore as melhores práticas e possa conter imprecisões..\n\n"
        f"User: {user_input}\n"
        "Assistant:"
    )


pairs = []
for instr in base_inputs:
    prompt_pos = build_prompt_positive(instr)
    prompt_neg = build_prompt_negative(instr)

    out_pos = generate(model, tokenizer, prompt_pos, max_new_tokens=200)
    out_neg = generate(model, tokenizer, prompt_neg, max_new_tokens=200)

    # RLCD: rótulo automático por construção (positivo é o chosen, negativo é o rejected)
    pairs.append({
        "prompt": instr,
        "chosen": out_pos,
        "rejected": out_neg,
        "meta": {
            "prompt_pos": prompt_pos,
            "prompt_neg": prompt_neg,
            "attribute": "helpfulness"
        }
    })

df = pd.DataFrame(pairs)
df

c:\dev\Track_RLAIF\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,prompt,chosen,rejected,meta
0,Explique para um leigo o que é uma vacina e po...,"Eu sei um vacina, que é um instrumento de vida...","Eu sei um leite de vacina, que é importante pa...",{'prompt_pos': 'Você é um assistente prestativ...
1,Qual país do mundo é um bom exemplo de democra...,"Um, um, um. Um, um. Um, um. Um, um. Um, um. Um...","Um, um, um, um, um, um, um, um, um, um, um, um...",{'prompt_pos': 'Você é um assistente prestativ...
2,Quais são os benefícios e riscos da inteligênc...,O sistema de informação e o sistema de científ...,O sistema de informação e o sistema de telefon...,{'prompt_pos': 'Você é um assistente prestativ...
